In [1]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

raw_train, raw_test = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[20%:]'],
    as_supervised=True
)

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.image import ResizeMethod

def preprocess(image, label):
    out_image = tf.image.resize(image, [224, 224], method=ResizeMethod.BICUBIC)
    out_image = tf.keras.applications.mobilenet_v2.preprocess_input(out_image)

    return out_image, label

batch_size = 32
train_batch = raw_train.map(preprocess).batch(batch_size)
test_batch = raw_test.map(preprocess).batch(batch_size)

In [3]:
mobilenet_base = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    weights="imagenet",
    include_top=False)

mobilenet_base.summary()

In [ ]:
mobilenet_base.trainable=False

mobilenet_model = tf.keras.Sequential([
    mobilenet_base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1)
])

mobilenet_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

mobilenet_model.fit(train_batch, epochs=5)
mobilenet_model.evaluate(test_batch, verbose=2)